In [7]:
import pandas as pd
import pulp

In [2]:
player = pd.read_csv('fplAnalytics-playerStautsData.csv')

In [3]:
# Filter to only available players
available = player[player['status'] == 'Available']

Cost effectiveness:
Money is a very limited resource in FPL. If I have all the money in the world, I'll have a Liverpool + Man City squad. To justify the investment, I'm calculating the point per cost ratio for the best value.

In [5]:
# Correct way to create the 'ppc' column
available.loc[:, 'ppc'] = available['total_points'] / available['cost']

In [8]:
# 1. Create the PuLP problem
prob = pulp.LpProblem("FPL_Optimization", pulp.LpMaximize)

# 2. Define the player variables
player_vars = pulp.LpVariable.dicts("player", available.index, cat='Binary')

# 3. Add the objective function
prob += pulp.lpSum([available.loc[i, 'points_per_game'] * player_vars[i] for i in available.index])

# 4. Add the constraints
#   a. Position constraints
prob += pulp.lpSum([player_vars[i] for i in available.index if available.loc[i, 'position'] == 'GKP']) == 1
prob += pulp.lpSum([player_vars[i] for i in available.index if available.loc[i, 'position'] == 'DEF']) == 3
prob += pulp.lpSum([player_vars[i] for i in available.index if available.loc[i, 'position'] == 'MID']) == 4
prob += pulp.lpSum([player_vars[i] for i in available.index if available.loc[i, 'position'] == 'FWD']) == 3

#   b. Cost constraint
prob += pulp.lpSum([available.loc[i, 'cost'] * player_vars[i] for i in available.index]) <= 83.4

#   c. Team constraint 
for team in available['team'].unique():
    prob += pulp.lpSum([player_vars[i] for i in available.index if available.loc[i, 'team'] == team]) <= 3 

#   d. Minutes constraint
for i in available.index:
    if available.loc[i, 'minutes'] < 1200:
        prob += player_vars[i] == 0  # Don't select players with less than 1200 minutes

# 5. Solve the problem
prob.solve()
print("Status:", pulp.LpStatus[prob.status])

# 6. Print the optimal solution (selected players with positions, sorted by position)
print("\nSelected Players (sorted by position):")

selected_players = []
for v in prob.variables():
    if v.varValue == 1:
        player_index = int(v.name.split('_')[1]) 
        player_name = available.loc[player_index, 'name']
        player_position = available.loc[player_index, 'position']
        selected_players.append((player_position, player_name))  # Store as tuples

selected_players.sort()  # Sort by position

for position, name in selected_players:
    print(f"{name} ({position})") 

#print toal points per game
print("\nTotal Points per Game:", pulp.value(prob.objective))

Status: Optimal

Selected Players (sorted by position):
Aina (DEF)
Alexander-Arnold (DEF)
Gabriel (DEF)
Isak (FWD)
Wissa (FWD)
Wood (FWD)
Sels (GKP)
Amad (MID)
Gordon (MID)
M.Salah (MID)
Mbeumo (MID)

Total Points per Game: 64.0
